In [ ]:
%%capture

import warnings

warnings.filterwarnings("ignore")
import _report_operator_section1_utils as section1
import _report_operator_section2_utils as section2
import altair as alt
import calitp_data_analysis.magics
import deploy_portfolio_yaml
import pandas as pd
import yaml
from IPython.display import HTML, Image, Markdown, display, display_html
from shared_utils import catalog_utils, rt_dates, rt_utils

with open("readable.yml") as f:
    readable_dict = yaml.safe_load(f)

In [ ]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [ ]:
# Comment out and leave this cell right below pandas
# portfolio_name = "Capitol Corridor Joint Powers Authority"
# portfolio_name = "Yurok Tribe"
# portfolio_name = "City of Santa Maria"
# portfolio_name = "San Diego Schedule"
portfolio_name = "City and County of San Francisco"

In [ ]:
# %%capture_parameters
# portfolio_name

In [ ]:
# Load crosswalk between portfolio_name vs organization_name
organization_name = deploy_portfolio_yaml.generate_operator_grain_yaml()

In [ ]:
organization_name = organization_name.loc[
    organization_name.portfolio_name == portfolio_name
]["organization_name"].iloc[0]

In [ ]:
# Load first dataset
operator_profiles = section1.load_operator_ntd_profile(organization_name)

In [ ]:
# Need to load "name" using this function rather than taking it from
# operator profiles above because "name" and "organization_name" differ.
names = section1.organization_name_crosswalk(organization_name)

In [ ]:
operator_route_map = section1.load_operator_map(names)

# Find percentiles
operator_route_map = section1.find_percentiles(operator_route_map, "Service Miles")

In [ ]:
scheduled_service = section1.load_operator_service_hours(names)

In [ ]:
# Dataset with agency metrics
try:
    agency_metrics_df = section2.load_operator_metrics(organization_name)
except:
    pass

In [ ]:
# Dataset for Section 2: because not every operator has realtime data,
# we need a try-except clause.
try:
    monthly_sched_vp_df, qtr_sched_vp_df = section2.load_schedule_vp_metrics(
        organization_name
    )
except:
    pass

In [ ]:
# Set drop down menu to be on the upper right for the charts
display(
    HTML(
        """
<style>
form.vega-bindings {
  position: absolute;
  right: 0px;
  top: 0px;
}
</style>
"""
    )
)

# {portfolio_name}

## Operator Overview

In [ ]:
try:
    date = operator_profiles["Date"].dt.to_pydatetime()[0]
except:
    pass

In [ ]:
try:
    service_area = int(operator_profiles.service_area_sq_miles.values[0])
    service_area = "{:,}".format(service_area)
    service_pop = int(operator_profiles.service_area_pop.values[0])
    service_pop = "{:,}".format(service_pop)
    display(
        Markdown(
            f"""{portfolio_name} is headquartered in <b>{operator_profiles.hq_city.values[0]}</b> in the Urbanized Area of <b>{operator_profiles.primary_uza_name.values[0]}</b>.<br>
            This operator provides <b>{service_area}</b> square miles of public transit service, which has a service population of <b>{service_pop}</b>.<br>
            This organization is a {operator_profiles.organization_type.values[0]}.<br>
            <b>Data Source</b>: <a href="https://www.transit.dot.gov/ntd/data-product/2022-annual-database-agency-information">National Transit Database</a> Annual Agency Information.
            """
        )
    )
except:
    pass

### Route Typologies

In [ ]:
try:
    display(
        Markdown(
            f"""The following data presents an overview of GTFS characteristics using data from the most recent date of
      <b>{date.strftime("%B")} {date.year}</b>."""
        )
    )
except:
    pass

In [ ]:
try:
    n_routes = operator_profiles["# Routes"].values[0]
    n_routes = "{:,}".format(n_routes)
    display(
        Markdown(
            f"""{portfolio_name} runs <b>{n_routes}</b> unique routes. Below is the breakdown of the routes and routes can belong to one or more categories.<p>
            Route categories are determined using a approach that looks at GTFS trips data
        alongside National Association of City Transportation Officials (NACTO)'s
        <a href="https://nacto.org/publication/transit-street-design-guide/introduction/service-context/transit-route-types/">Transit Route Types</a> 
        and <a href= "https://nacto.org/publication/transit-street-design-guide/introduction/service-context/transit-frequency-volume/">Frequency and Volume</a>
        guides. Please see the <a href="https://github.com/cal-itp/data-analyses/blob/main/gtfs_digest/methodology.md">methodology docs</a> for more details on this approach.
        """
        )
    )
except:
    display(Markdown(f"""{portfolio_name} doesn't have an operator profile."""))

In [ ]:
try:
    route_typology_df = section1.route_typology(operator_profiles)
    display(
        section1.basic_pie_chart(
            route_typology_df,
            "route_type",
            "total_routes",
            readable_dict["route_typology"]["title"],
            readable_dict["route_typology"]["subtitle"],
        )
    )

except:
    display(
        Markdown(f"""{portfolio_name} doesn't have route typology data available.""")
    )

### Service Area

In [ ]:
try:
    n_service_miles = int(operator_profiles["Operator Service Miles"].values[0])
    n_service_miles = "{:,}".format(n_service_miles)

    avg_service_miles = int(operator_route_map["Service Miles"].mean())
    avg_service_miles = "{:,}".format(avg_service_miles)

    route_percentile_agg = section1.route_length_percentile(operator_route_map)

    display(
        Markdown(
            f"""{portfolio_name} provides <b>{n_service_miles}</b> miles of public transit.
        The average length of a route is <b>{avg_service_miles}</b> miles.<p> 
        """
        )
    )

    display(
        section1.basic_bar_chart(
            route_percentile_agg,
            "Total Unique Routes",
            "percentile_group",
            readable_dict["route_percentiles"]["title"],
            readable_dict["route_percentiles"]["subtitle"],
        )
    )

    longest_shortest_df = section1.shortest_longest_route(operator_route_map)
    display(
        section1.basic_bar_chart(
            longest_shortest_df,
            "Service Miles",
            "Route",
            readable_dict["longest_shortest_route"]["title"],
            readable_dict["longest_shortest_route"]["subtitle"],
        )
    )
except:
    pass

In [ ]:
try:
    n_stops = int(operator_profiles["# Stops"].values[0])
    n_stops = "{:,}".format(n_stops)

    n_arrivals = int(operator_profiles["# Arrivals"].values[0])
    n_arrivals = "{:,}".format(n_arrivals)

    mean_arrivals = int(operator_profiles["Avg Arrivals per Stop"].values[0])
    mean_arrivals = "{:,}".format(mean_arrivals)
    display(
        Markdown(
            f"""{portfolio_name} stops at <b>{n_stops}</b> unique stops, 
            totaling to <b>{n_arrivals}</b> arrivals. The average arrivals per stop 
            is <b>{mean_arrivals}</b>.
        """
        )
    )
except:
    pass

In [ ]:
try:
    counties_df = section1.counties_served(operator_route_map)
    display(
        Markdown(
            f"{portfolio_name} provides public transit in {counties_df['all'].values[0]} counties."
        )
    )
except:
    display(Markdown(f"""{portfolio_name} doesn't have an route geographies."""))

### Map of Routes

In [ ]:
try:
    display(
        operator_route_map.explore(
            "Route",
            cmap="Spectral",
            tiles="CartoDB positron",
            width=500,
            height=300,
            style_kwds={"weight": 3},
            legend=False,
            tooltip=["Route", "Service Miles"],
        )
    )
except:
    display(Markdown(f"""{portfolio_name} doesn't have an route geographies."""))

### Service Hours 

In [ ]:
try:
    display(
        section1.create_service_hour_chart(
            scheduled_service,
            "Weekday",
            "Weekday Service Hours",
            readable_dict["daily_scheduled_hour"]["subtitle"],
        )
    )
except:
    pass

In [ ]:
try:
    display(
        section1.create_service_hour_chart(
            scheduled_service, "Saturday", "Weekend Service Hours", ""
        )
    )
except:
    pass

In [ ]:
try:
    display(
        section1.create_service_hour_chart(
            scheduled_service, "Sunday", "Weekend Service Hours", ""
        )
    )
except:
    pass

## Metrics for All Routes 

In [ ]:
if (agency_metrics_df["VP per Minute (All Routes)"].isna().sum() > 2) | (
    agency_metrics_df["Spatial Accuracy (All Routes)"].isna().sum() > 2
):
    display(Markdown(f"""{organization_name} only has schedule data."""))
else:
    display(section2.agency_overview(agency_metrics_df))

## Detailed Route Overview

In [ ]:
try:
    display(section2.filtered_route(monthly_sched_vp_df, qtr_sched_vp_df))
except:
    display(Markdown(f"""{portfolio_name} only has schedule data."""))